In [1]:
#import tuxml
import pandas as pd 
import warnings
warnings.filterwarnings('ignore')

#definitions
# systems = ['7z','BerkeleyDBC','Dune','Hipacc','Irzip','LLVM','Polly','x264']
# systems = ['LLVM','x264','BerkeleyDBC','Irzip','Polly','7z','Hipacc','Dune']
systems = ['Dune']

# samples_config = ['random']
samples_config = ['distBased', 'divDistBased','henard', 'random', 'solverBased', 'twise']
size_methods = ["Performance"]


In [2]:
#learning a model by using linear regression
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import ElasticNet

def run_regressorML(reg, test_size, size_target, x_train, x_test, y_train, y_test):
    assert(size_target in size_methods)
    reg.fit(x_train, y_train)       
    y_pred = reg.predict(x_test)
    #y_pred = reg.intercept_ + np.sum(reg.coef_ * x_test.values, axis=1)
    dfErrors = pd.DataFrame({'Actual':y_test, 'Predicted':y_pred, "error":(y_pred - y_test).abs(), "% error":((y_pred - y_test)/y_test).abs()*100})
    return dfErrors["% error"].describe()

In [3]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import tree
import pandas as pd

def ft_importances(coef_, col):
    importanceSeries = pd.Series(coef_, index=col.values)
    return importanceSeries[importanceSeries != 0].abs().sort_values(ascending = False)

def ft_importances_RF(X, y):
    model = RandomForestRegressor(random_state=42)
    model.fit(X, y)

    feature_importances = model.feature_importances_
    features = pd.DataFrame({'feature': X.columns, 'importance': feature_importances})
    features.sort_values('importance', ascending=False, inplace=True)
    
    return features

def ft_importances_DT(X, y):
    model = tree.DecisionTreeRegressor(random_state=42)
    model.fit(X, y)

    feature_importances = model.feature_importances_
    features = pd.DataFrame({'feature': X.columns, 'importance': feature_importances})
    features.sort_values('importance', ascending=False, inplace=True)
    
    return features

def ft_importances_GB(X, y):
    model = GradientBoostingRegressor(random_state=42)
    model.fit(X, y)

    feature_importances = model.feature_importances_
    features = pd.DataFrame({'feature': X.columns, 'importance': feature_importances})
    features.sort_values('importance', ascending=False, inplace=True)
    
    return features


In [4]:
df_result  = pd.DataFrame(columns = ['algorithm_name', 'ft_dummyfication', 'origin_ft_selection', 'ft_selection', 'hyperparameters', 'size_target', 'test_size', 'coef_order', 'accuracy'])
df_result

,algorithm_name,ft_dummyfication,origin_ft_selection,ft_selection,hyperparameters,size_target,test_size,coef_order,accuracy


In [5]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import tree
from sklearn import linear_model
from sklearn import metrics
from sklearn import ensemble
from IPython.core.debugger import set_trace

%run feature_importance-all_systems.ipynb


def run_Algorithms(df, ft_importance_enable, system=None):
    
    samples_config = ['distBased', 'divDistBased','henard', 'random', 'solverBased', 'twise']
    size_methods = ["Performance"]
    
    for config_name in samples_config:
        
        df_result = pd.DataFrame(columns = ['algorithm_name', 'ft_dummyfication', 'origin_ft_selection', 'ft_selection', 'hyperparameters', 'size_target', 'test_size', 'coef_order', 'accuracy', 'num_features'])
        
        print(config_name)
        path_name = config_name+"_t3"
        if(system==None):
            path = r"sampledConfigurations_"+path_name+".csv"
        else:
            path = system+"/sampledConfigurations_"+path_name+".csv"
            
        if not ft_importance_enable:
            samplePerc_lst = [1.0]
        else:
            samplePerc_lst = [0.3,0.4,0.5,0.6]
            
        for samplePerc in samplePerc_lst:
            print(samplePerc)
            df = pd.read_csv(path, sep=';')
            
            if ft_importance_enable:
                #Run fetures engines: #yes and encoding
                df = run_feature_encoding(run_features_engine_yes(df))

                path_name = config_name+"_t3_features_engine"
            else:
                df = run_features_engine_yes(df)

            
            #display(df)
            #number of features after features engine execution
            num_features = round(samplePerc*len(df.columns))

            start_time_sample = pd.Timestamp.now()
            
            
            #Run feature selection
            if ft_importance_enable:
                run_featureSelection(df, system, None, path_name,'RandomForest')
                df_importance = pd.read_csv(system+f"/feature_importance_{path_name}_RF.csv")

                print("Sample features number: "+ str(num_features))
                df_importance = df[df_importance[:num_features]["Unnamed: 0"].values]
                df_importance.head()


            target = "Performance"
            paramsLasso = {'alpha': [25,10,4,2,1.0,0.8,0.5,0.3,0.2,0.1,0.05,0.02,0.01],
                           'max_iter': [1, 5, 10, 100, 1000],}
            paramsRidge = {'alpha': [25,10,4,2,1.0,0.8,0.5,0.3,0.2,0.1,0.05,0.02,0.01]}
            paramsENet = {'alpha': [25,10,4,2,1.0,0.8,0.5,0.3,0.2,0.1,0.05,0.02,0.01],
                          'l1_ratio': [0.0, 0.5, 1.0, 0.1],
                          'max_iter': [1, 5, 10, 100, 1000],}
            paramsRF =  {"max_depth":18,"n_estimators":50,"verbose":2}

            #Modifiquei os dicionários para quando a feature importance está ativada. Acho que não fazia muito sentido 
            #manter o uso do random forest quando feature importance for False, da mesma forma que não é viável manter
            #o None quando está ativado
            
            if ft_importance_enable:
                ft_selection =  {'DecisionTree': df_importance.columns[:num_features]}
            else:
                ft_selection =  {'None': df.drop(columns=size_methods).columns}

            for origin_ft_selection in ft_selection:
                print(origin_ft_selection)
                if ft_importance_enable:
                    print('IN FT_LOOP DF IMPORTANCE\n')
                    display(df_importance)

                for size in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]:
                    config_start_time = pd.Timestamp.now()

                    # size = number of configurations
                    x_train, x_test, y_train, y_test = train_test_split(df[ft_selection[origin_ft_selection]], df[target], train_size=size, random_state=42)
                    x_train, y_train = np.array(x_train.values), np.array(y_train.values)


                    if len(x_train) > 3:

                        model = {'LinearRegression': LinearRegression(),
                                 'Lasso': GridSearchCV(linear_model.Lasso(), param_grid=paramsLasso, cv=3).fit(x_train, y_train).best_estimator_,
                                 'Ridge': GridSearchCV(linear_model.Ridge(), param_grid=paramsRidge, cv=3).fit(x_train, y_train).best_estimator_,
                                 'ElasticNet': GridSearchCV(ElasticNet(), param_grid=paramsENet, scoring='r2', cv=3).fit(x_train, y_train).best_estimator_,
                                 'PolynomialRegression': LinearRegression(),
                                 'RandomForest': ensemble.RandomForestRegressor(),
                                 'GradientBoostingTree': ensemble.GradientBoostingRegressor(),
                                 'DecisionTree': tree.DecisionTreeRegressor()}
                    else:
                        if len(x_train) >= 2:
                            model = {'LinearRegression': LinearRegression(),
                                     'Lasso': GridSearchCV(linear_model.Lasso(), param_grid=paramsLasso, cv=2).fit(x_train, y_train).best_estimator_,
                                     'Ridge': GridSearchCV(linear_model.Ridge(), param_grid=paramsRidge, cv=2).fit(x_train, y_train).best_estimator_,
                                     'ElasticNet': GridSearchCV(ElasticNet(), param_grid=paramsENet, scoring='r2', cv=2).fit(x_train, y_train).best_estimator_,
                                     'PolynomialRegression': LinearRegression(),
                                     'RandomForest': ensemble.RandomForestRegressor(),
                                     'GradientBoostingTree': ensemble.GradientBoostingRegressor(),
                                     'DecisionTree': tree.DecisionTreeRegressor()}

                    for key in model:
                        print(key)
                        if (key == 'PolynomialRegression') and (origin_ft_selection == 'None'):
                            print("It doesn't scale")
                        else:
                            if (key == 'PolynomialRegression'):
                                #for 2 options we have: 𝑓(𝑥₁, 𝑥₂) = 𝑏₀ + 𝑏₁𝑥₁ + 𝑏₂𝑥₂ + 𝑏₃𝑥₁² + 𝑏₄𝑥₁𝑥₂ + 𝑏₅𝑥₂²
                                x_train = PolynomialFeatures(degree=2, include_bias=False).fit_transform(x_train)
                                x_test = PolynomialFeatures(degree=2, include_bias=False).fit_transform(x_test)

                            res_model = run_regressorML(model[key], test_size=size, size_target=target, x_train=x_train, x_test=x_test, y_train=y_train, y_test=y_test)

                            if (key == 'PolynomialRegression'):
                                model[key].coef_ = model[key].coef_[:num_features]
                                
                            if (key == 'DecisionTree' or key == 'RandomForest' or key == 'GradientBoostingTree'):
                                coef_order = ft_importances(res_model[:num_features], col=ft_selection[origin_ft_selection])
                            else:
                                coef_order = ft_importances(model[key].coef_, col=ft_selection[origin_ft_selection])

#                             df_result.loc[len(df_result)] = [key, ft_dummyfication, origin_ft_selection, ft_selection[origin_ft_selection], model[key], target, size, coef_order, res_model]
                            df_result.loc[len(df_result)] = [key, False, origin_ft_selection, ft_selection[origin_ft_selection], model[key], target, size, coef_order, res_model, str(num_features)]
                            display(df_result)
                            
                        config_end_time = pd.Timestamp.now()
                        exec_time = [config_name,samplePerc,num_features,ft_importance_enable,origin_ft_selection, key, size,False,config_start_time,config_end_time]
    #             exec_time = [config_name,samplePerc,num_features,ft_importance_enable,origin_ft_selection, key, size,ft_dummyfication,config_start_time,config_end_time]
                        df_time.loc[df_time.size] = exec_time

            sample_end_time = pd.Timestamp.now()
            exec_time = [config_name,samplePerc,num_features,ft_importance_enable,origin_ft_selection, key, None,None,start_time_sample,sample_end_time]
            df_time.loc[df_time.size] = exec_time
#             display(df_result)

            if ft_importance_enable:
                df_result.to_csv("results/"+system+"/"+system+"-results_with_feature_importance_"+path_name+"_"+str(num_features)+"_v3_RF.csv", header=True)
            else:
                df_result.to_csv("results/"+system+"/"+system+"-results_"+path_name+"_full_v3_RF.csv", header=True)

        display(df_result)
                                                                                                                                  
    if ft_importance_enable:
        time_file_name = "results/"+system+"/"+system+"-simulation_time_information-with_feature_importance_v3_RF.csv"
    else:
        time_file_name = "results/"+system+"/"+system+"-simulation_time_information_v3_RF.csv"
    
    df_time.to_csv(time_file_name)
        
    return df_result

In [ ]:
df_time = pd.DataFrame(columns=['SampleAlgorithm','%FeatureSelection','#Features','FeatureImportance','OriginalFS','Algorithm','%Configurations','Dummy','StartTime', 'EndTime'])

for system in systems:
    print(system)
    
    print("\n\n-----------------------------------------------------------------------------")
    print("\n--------------- System "+ system+ " training WITHOUT features importance ")
    print("\n--------------- Start time:: " + str(pd.Timestamp.now()))
    print("\n-----------------------------------------------------------------------------")
    df_result_nfi = run_Algorithms(None, False, system)
    print("\n\n-----------------------------------------------------------------------------")
    print("\n--------------- System "+ system+ " training ")
    print("\n--------------- End time:: " + str(pd.Timestamp.now()))
    print("\n-----------------------------------------------------------------------------")
    
    print("\n\n-----------------------------------------------------------------------------")
    print("\n--------------- System "+ system+ " training WITH features importance ")
    print("\n--------------- Start time:: " + str(pd.Timestamp.now()))
    print("\n-----------------------------------------------------------------------------")
    df_result_fi = run_Algorithms(None, True, system)
    print("\n\n-----------------------------------------------------------------------------")
    print("\n--------------- System "+ system+ " training ")
    print("\n--------------- End time:: " + str(pd.Timestamp.now()))
    print("\n-----------------------------------------------------------------------------")

    

Dune


-----------------------------------------------------------------------------

--------------- System Dune training WITHOUT features importance 

--------------- Start time:: 2024-03-30 16:07:22.038856

-----------------------------------------------------------------------------
distBased
1.0
None
LinearRegression


,algorithm_name,ft_dummyfication,origin_ft_selection,ft_selection,hyperparameters,size_target,test_size,coef_order,accuracy,num_features
0,LinearRegression,False,None,"Index(['root', 'Precon', 'Solver', 'SeqGS', 'S...",LinearRegression(),Performance,0.1,Precon 9.227860e+17 CGSolver ...,count 964.000000 mean 22.546150 std ...,34


Lasso


,algorithm_name,ft_dummyfication,origin_ft_selection,ft_selection,hyperparameters,size_target,test_size,coef_order,accuracy,num_features
0,LinearRegression,False,None,"Index(['root', 'Precon', 'Solver', 'SeqGS', 'S...",LinearRegression(),Performance,0.1,Precon 9.227860e+17 CGSolver ...,count 964.000000 mean 22.546150 std ...,34
1,Lasso,False,None,"Index(['root', 'Precon', 'Solver', 'SeqGS', 'S...","Lasso(alpha=25, max_iter=1)",Performance,0.1,pre_1 8.639988e+03 post_0 ...,count 964.000000 mean 23.822746 std ...,34


Ridge


...

,algorithm_name,ft_dummyfication,origin_ft_selection,ft_selection,hyperparameters,size_target,test_size,coef_order,accuracy,num_features
0,LinearRegression,False,None,"Index(['root', 'Precon', 'Solver', 'SeqGS', 'S...",LinearRegression(),Performance,0.1,Precon 9.227860e+17 CGSolver ...,count 964.000000 mean 22.546150 std ...,34
1,Lasso,False,None,"Index(['root', 'Precon', 'Solver', 'SeqGS', 'S...","Lasso(alpha=25, max_iter=1)",Performance,0.1,pre_1 8.639988e+03 post_0 ...,count 964.000000 mean 23.822746 std ...,34
2,Ridge,False,None,"Index(['root', 'Precon', 'Solver', 'SeqGS', 'S...",Ridge(alpha=10),Performance,0.1,post_0 3874.625270 pre_1 ...,count 964.000000 mean 15.804887 std ...,34
3,ElasticNet,False,None,"Index(['root', 'Precon', 'Solver', 'SeqGS', 'S...","ElasticNet(alpha=0.2, l1_ratio=0.1, max_iter=5)",Performance,0.1,post_0 2820.216079 pre_1 ...,count 964.000000 mean 14.651719 std ...,34
4,RandomForest,False,None,"Index(['root', 'Precon', 'Solver', 'SeqGS', 'S...","(DecisionTreeRegressor(max_features=1.0, rando...",Performance,0.1,root NaN Precon NaN Solv...,count 964.000000 mean 10.375541 std ...,34
5,GradientBoostingTree,False,None,"Index(['root', 'Precon', 'Solver', 'SeqGS', 'S...",([DecisionTreeRegressor(criterion='friedman_ms...,Performance,0.1,root NaN Precon NaN Solv...,count 964.000000 mean 8.691252 std ...,34
6,DecisionTree,False,None,"Index(['root', 'Precon', 'Solver', 'SeqGS', 'S...",DecisionTreeRegressor(),Performance,0.1,root NaN Precon NaN Solv...,count 964.000000 mean 9.910599 std ...,34
7,LinearRegression,False,None,"Index(['root', 'Precon', 'Solver', 'SeqGS', 'S...",LinearRegression(),Performance,0.2,LoopSolver 5.784718e+17 BiCGSTABSolver ...,count 857.000000 mean 17.893303 std ...,34
8,Lasso,False,None,"Index(['root', 'Precon', 'Solver', 'SeqGS', 'S...","Lasso(alpha=25, max_iter=5)",Performance,0.2,post_0 6074.699877 pre_1 ...,count 857.000000 mean 16.627685 std ...,34
9,Ridge,False,None,"Index(['root', 'Precon', 'Solver', 'SeqGS', 'S...",Ridge(alpha=10),Performance,0.2,post_0 3697.317425 pre_1 ...,count 857.000000 mean 14.976374 std ...,34


GradientBoostingTree


,algorithm_name,ft_dummyfication,origin_ft_selection,ft_selection,hyperparameters,size_target,test_size,coef_order,accuracy,num_features
0,LinearRegression,False,None,"Index(['root', 'Precon', 'Solver', 'SeqGS', 'S...",LinearRegression(),Performance,0.1,Precon 9.227860e+17 CGSolver ...,count 964.000000 mean 22.546150 std ...,34
1,Lasso,False,None,"Index(['root', 'Precon', 'Solver', 'SeqGS', 'S...","Lasso(alpha=25, max_iter=1)",Performance,0.1,pre_1 8.639988e+03 post_0 ...,count 964.000000 mean 23.822746 std ...,34
2,Ridge,False,None,"Index(['root', 'Precon', 'Solver', 'SeqGS', 'S...",Ridge(alpha=10),Performance,0.1,post_0 3874.625270 pre_1 ...,count 964.000000 mean 15.804887 std ...,34
3,ElasticNet,False,None,"Index(['root', 'Precon', 'Solver', 'SeqGS', 'S...","ElasticNet(alpha=0.2, l1_ratio=0.1, max_iter=5)",Performance,0.1,post_0 2820.216079 pre_1 ...,count 964.000000 mean 14.651719 std ...,34
4,RandomForest,False,None,"Index(['root', 'Precon', 'Solver', 'SeqGS', 'S...","(DecisionTreeRegressor(max_features=1.0, rando...",Performance,0.1,root NaN Precon NaN Solv...,count 964.000000 mean 10.375541 std ...,34
5,GradientBoostingTree,False,None,"Index(['root', 'Precon', 'Solver', 'SeqGS', 'S...",([DecisionTreeRegressor(criterion='friedman_ms...,Performance,0.1,root NaN Precon NaN Solv...,count 964.000000 mean 8.691252 std ...,34
6,DecisionTree,False,None,"Index(['root', 'Precon', 'Solver', 'SeqGS', 'S...",DecisionTreeRegressor(),Performance,0.1,root NaN Precon NaN Solv...,count 964.000000 mean 9.910599 std ...,34
7,LinearRegression,False,None,"Index(['root', 'Precon', 'Solver', 'SeqGS', 'S...",LinearRegression(),Performance,0.2,LoopSolver 5.784718e+17 BiCGSTABSolver ...,count 857.000000 mean 17.893303 std ...,34
8,Lasso,False,None,"Index(['root', 'Precon', 'Solver', 'SeqGS', 'S...","Lasso(alpha=25, max_iter=5)",Performance,0.2,post_0 6074.699877 pre_1 ...,count 857.000000 mean 16.627685 std ...,34
9,Ridge,False,None,"Index(['root', 'Precon', 'Solver', 'SeqGS', 'S...",Ridge(alpha=10),Performance,0.2,post_0 3697.317425 pre_1 ...,count 857.000000 mean 14.976374 std ...,34


DecisionTree


,algorithm_name,ft_dummyfication,origin_ft_selection,ft_selection,hyperparameters,size_target,test_size,coef_order,accuracy,num_features
0,LinearRegression,False,None,"Index(['root', 'Precon', 'Solver', 'SeqGS', 'S...",LinearRegression(),Performance,0.1,Precon 9.227860e+17 CGSolver ...,count 964.000000 mean 22.546150 std ...,34
1,Lasso,False,None,"Index(['root', 'Precon', 'Solver', 'SeqGS', 'S...","Lasso(alpha=25, max_iter=1)",Performance,0.1,pre_1 8.639988e+03 post_0 ...,count 964.000000 mean 23.822746 std ...,34
2,Ridge,False,None,"Index(['root', 'Precon', 'Solver', 'SeqGS', 'S...",Ridge(alpha=10),Performance,0.1,post_0 3874.625270 pre_1 ...,count 964.000000 mean 15.804887 std ...,34
3,ElasticNet,False,None,"Index(['root', 'Precon', 'Solver', 'SeqGS', 'S...","ElasticNet(alpha=0.2, l1_ratio=0.1, max_iter=5)",Performance,0.1,post_0 2820.216079 pre_1 ...,count 964.000000 mean 14.651719 std ...,34
4,RandomForest,False,None,"Index(['root', 'Precon', 'Solver', 'SeqGS', 'S...","(DecisionTreeRegressor(max_features=1.0, rando...",Performance,0.1,root NaN Precon NaN Solv...,count 964.000000 mean 10.375541 std ...,34
5,GradientBoostingTree,False,None,"Index(['root', 'Precon', 'Solver', 'SeqGS', 'S...",([DecisionTreeRegressor(criterion='friedman_ms...,Performance,0.1,root NaN Precon NaN Solv...,count 964.000000 mean 8.691252 std ...,34
6,DecisionTree,False,None,"Index(['root', 'Precon', 'Solver', 'SeqGS', 'S...",DecisionTreeRegressor(),Performance,0.1,root NaN Precon NaN Solv...,count 964.000000 mean 9.910599 std ...,34
7,LinearRegression,False,None,"Index(['root', 'Precon', 'Solver', 'SeqGS', 'S...",LinearRegression(),Performance,0.2,LoopSolver 5.784718e+17 BiCGSTABSolver ...,count 857.000000 mean 17.893303 std ...,34
8,Lasso,False,None,"Index(['root', 'Precon', 'Solver', 'SeqGS', 'S...","Lasso(alpha=25, max_iter=5)",Performance,0.2,post_0 6074.699877 pre_1 ...,count 857.000000 mean 16.627685 std ...,34
9,Ridge,False,None,"Index(['root', 'Precon', 'Solver', 'SeqGS', 'S...",Ridge(alpha=10),Performance,0.2,post_0 3697.317425 pre_1 ...,count 857.000000 mean 14.976374 std ...,34


In [ ]:
df_result['accuracy'][0]

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

rs = []
for i in range(len(df_result_fi['accuracy'])):
    rs.append(df_result_fi['accuracy'][i]['mean'])

# rs
# display (df_result['algorithm_name'], df_result['accuracy'], rs)

plt.scatter(df_result_fi['algorithm_name'], rs, color ="blue", marker = "s")
plt.plot(df_result_fi['algorithm_name'], rs, color ="blue")


In [ ]:
# Boxplot
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd


data = [model['LinearRegression'].coef_, model['Lasso'].coef_, model['Ridge'].coef_, model['ElasticNet'].coef_, model['PolynomialRegression'].coef_]
fig1, ax1 = plt.subplots()
ax1.set_title('Coeficient of feature importance for all algorithms')
ax1.boxplot(data)
plt.show()